In [2]:
import os
from configparser import ConfigParser
path_config_file = os.getcwd().split('docrv2_sroie')[0] + 'docrv2_sroie/' + 'config.ini'
common_variable = ConfigParser()
common_variable.read(path_config_file)

print(common_variable.sections())
print(common_variable['database']['module_path'])
print(common_variable['database']['DB_BASE'])

import sys
sys.path.append(common_variable['database']['module_path'])
from db_util import *
from db import *

import tensorflow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy

from parse import parse
from PIL import Image, ImageFont, ImageDraw, ImageEnhance
from glob import glob
from tqdm import tqdm_notebook

['database', 'mlflow', 'font']
/media/myungsungkwak/msdisk/docrv2_sroie/modules/database/
/media/myungsungkwak/msdisk/docrv2_data/DB/


In [6]:
check_db_name = "d2gan_raw_training_halfsizev2"
check_db_path = common_variable['database']['DB_BASE']
check_db = open_env(os.path.join(check_db_path, check_db_name))
print_env(check_db)
#check_imgs = read_bulk_data_from_db(check_db, prefix='img')

+-------------+----------------------------------------------------------+
| Key         | Value                                                    |
+-------------+----------------------------------------------------------+
| class       | train                                                    |
| created     | 2020-09-14 10:36:48                                      |
| db_data     | (database)                                               |
| description | datadb_d2gan, resized to 1/2, random crop, 50im per data |
| name        | d2gan_raw_training_halfsizev2                            |
| note        |                                                          |
| reference   | /home/dk/docrv2_sroie/DB/d2gan_raw_training_halfsizev2   |
| sep         | 	                                                        |
| updated     | 2020-09-14 10:36:48                                      |
+-------------+----------------------------------------------------------+


In [7]:
cache={}
cache[b'reference'] = str(os.path.join(check_db_path, check_db_name)).encode()
write_cache_to_env(check_db, cache)

In [8]:
print_env(check_db)

+-------------+--------------------------------------------------------------------------+
| Key         | Value                                                                    |
+-------------+--------------------------------------------------------------------------+
| class       | train                                                                    |
| created     | 2020-09-14 10:36:48                                                      |
| db_data     | (database)                                                               |
| description | datadb_d2gan, resized to 1/2, random crop, 50im per data                 |
| name        | d2gan_raw_training_halfsizev2                                            |
| note        |                                                                          |
| reference   | /media/myungsungkwak/msdisk/docrv2_data/DB/d2gan_raw_training_halfsizev2 |
| sep         | 	                                                                        |

In [ ]:
def create_database(db_idx, db_name=None):
    path = common_variable['database']['DB_BASE']
    os.makedirs(path, exist_ok=True)

    name = db_name + "_%04d"%db_idx
    description = 'data db from train_task2 all coords images'
    datadb = create_data_db(path=path, name=name, description=description, data_class='DATA')
    print("... created db : ", name)
    return datadb

def update_database(datadb, imgs, coords, refname, db_start_idx):

    for idx, img in enumerate(imgs):
        chars = []
        transpose_chars = np.transpose(coords[idx])[8]
        transpose_chars = list(map(str, transpose_chars))
        chars.append(transpose_chars)


        char_table = pd.DataFrame(coords[idx])
        char_table.rename(columns={0:'x1', 1:'y1', 2:'x2', 3:'y2', 4:'x3',
                                   5:'y3', 6:'x4', 7:'y4', 8:'char'}, inplace=True)
        char_table.x1 = char_table.x1.astype(int)
        char_table.y1 = char_table.y1.astype(int)
        char_table.x2 = char_table.x2.astype(int)
        char_table.y2 = char_table.y2.astype(int)
        char_table.x3 = char_table.x3.astype(int)
        char_table.y3 = char_table.y3.astype(int)
        char_table.x4 = char_table.x4.astype(int)
        char_table.y4 = char_table.y4.astype(int)
        
        update_data(datadb, 
                    index=db_start_idx+idx, 
                    label='task12', 
                    image=img, 
                    text=None, 
                    ref=refname, 
                    char_c=char_table,
                    char=chars[0])

def get_coords_from_image(txt_file):
    lps = []

    try:
        with open(txt_file, 'r') as f:
            coords_data = f.read().split('\n')
            for ii in range(len(coords_data)):
                if len(coords_data[ii]) > 0:
                    ps = parse('{},{},{},{},{},{},{},{},{}', coords_data[ii])
                    ps = list(ps)
                    ps[:-1] = list(map(int, ps[:-1]))
                    lps.append(ps)
    except:
        with open(txt_file, 'r', encoding='ISO-8859-1') as f:
            coords_data = f.read().split('\n')
            for ii in range(len(coords_data)):
                if len(coords_data[ii]) > 0:
                    ps = parse('{},{},{},{},{},{},{},{},{}', coords_data[ii])
                    ps = list(ps)
                    ps[:-1] = list(map(int, ps[:-1]))
                    lps.append(ps)
    return lps

In [82]:
"""
method 1 - no padding
crop images followed as original x1 coordiantes (x1 random choice)
crop size 256, 128
"""

def get_crop_image(img_path, coords, crop_size=(256,128), num_images=1):
    
    img = Image.open(img_path)
    o_width = img.size[0]
    o_height = img.size[1]

    transpose_x1_coords = np.transpose(coords)[0]
    transpose_y1_coords = np.transpose(coords)[1]
    transpose_x1_coords = list(map(int, transpose_x1_coords)) 
    transpose_y1_coords = list(map(int, transpose_y1_coords)) 

    sample_list = []
    coords_list = []
    for idx in range(num_images):
        while True:
            # base_idx = idx # (all x1 coords)
            base_idx = np.random.choice(len(transpose_x1_coords), 1, replace=False)[0]
            
            base_x1 = transpose_x1_coords[base_idx]
            base_y1 = transpose_y1_coords[base_idx]

            new_x3 = base_x1 + crop_size[0]
            new_y3 = base_y1 + crop_size[1]

            if (new_x3 > o_width) or (new_y3 > o_height):
                continue
            crop_img = img.crop((base_x1, base_y1, new_x3, new_y3))
            copy_coords = copy.deepcopy(coords)
            crop_coords = calculate_coords(copy_coords, base_idx, (base_x1, base_y1))
            
            coords_list.append(crop_coords)
            sample_list.append(crop_img)
            del crop_coords[:][:]
            break
    return sample_list, coords_list, base_idx, (base_x1,base_y1)

In [ ]:
"""
method 2 - random padding
crop size 256, 128
"""
def get_crop_image_by_coords_pad_random(img_path, coords, crop_size=(256,128), num_images=1):
    
    img = Image.open(img_path)
    
    o_width = img.size[0]
    o_height = img.size[1]

    transpose_x1_coords = np.transpose(coords)[0]
    transpose_y1_coords = np.transpose(coords)[1]
    transpose_x1_coords = list(map(int, transpose_x1_coords)) 
    transpose_y1_coords = list(map(int, transpose_y1_coords))
    
    transpose_x2_coords = np.transpose(coords)[2]
    transpose_y2_coords = np.transpose(coords)[3]
    transpose_x2_coords = list(map(int, transpose_x2_coords))
    transpose_y2_coords = list(map(int, transpose_y2_coords))

    transpose_x3_coords = np.transpose(coords)[4]
    transpose_y3_coords = np.transpose(coords)[5]
    transpose_x3_coords = list(map(int, transpose_x3_coords))
    transpose_y3_coords = list(map(int, transpose_y3_coords))

    transpose_x4_coords = np.transpose(coords)[6]
    transpose_y4_coords = np.transpose(coords)[7]
    transpose_x4_coords = list(map(int, transpose_x4_coords))
    transpose_y4_coords = list(map(int, transpose_y4_coords))

    transpose_chars = np.transpose(coords)[8]

    sample_list = []
    coords_list = []
    img_cnt = 0
    
    for _ in range(np.random.randint(1, 10)):
        # num of coords
        for idx in range(num_images):
            while True:
                base_idx = idx
                pdd = np.random.randint(-5,10,2) # [x1, y1]
                
                base_x1 = transpose_x1_coords[base_idx]
                base_y1 = transpose_y1_coords[base_idx]
                
                new_x1 = base_x1 - pdd[0]
                new_y1 = base_y1 - pdd[1]
                new_x3 = base_x1 + crop_size[0] - pdd[0]
                new_y3 = base_y1 + crop_size[1] - pdd[1]
                
                if (new_x1 < 0) or (new_y1 < 0):
                    continue
                if (new_x3 > o_width) or (new_y3 > o_height):
                    if (base_x1 + crop_size[0]) > o_width or (base_y1 + crop_size[1]) > o_height:
                        break
                    else:
                        continue

                crop_img = img.crop((new_x1, new_y1, new_x3, new_y3))
                #print("- {}, {}, pdd: {} --> {}".format(idx, crop_img.size, pdd, transpose_chars[idx]))

                copy_coords = copy.deepcopy(coords)
                crop_coords = calculate_coords(copy_coords, base_idx, (new_x1, new_y1), pdd=(pdd[0],pdd[1]))

                coords_list.append(crop_coords)
                sample_list.append(crop_img)

                img_cnt = img_cnt +1

                del crop_coords[:][:]
                break
    return sample_list, coords_list, img_cnt

In [ ]:
"""
run1: no padding (use get_crop_image)
"""
datadir = '/media/myungsungkwak/msdisk/docrv2_sroie/data/train_task1_crop/'
img_files = glob(datadir+'*.jpg')
txt_files = glob(datadir+'*.txt')

crop_images = []
coords_table = []

print(len(img_files))

datadb = create_database(100, "d2gan_train_all")
num_images = 5
db_start_idx = 0
#for idx in tqdm(range(len(img_files))):
for idx in tqdm(range(2)):
    # print(idx, " : ", img_files[idx])
    refname = os.path.basename(img_files[idx])[:-4]
    txt_filepath = os.path.join(datadir, refname+'.txt')
    coords_table.append(get_coords_from_image(txt_filepath))
    crop_img, crop_coords, crop_idx, crop_xy = get_crop_image(img_files[idx],
                                                              coords_table[idx],
                                                              crop_size=(256,128),
                                                              num_images=num_images)
    update_database(datadb, crop_img, crop_coords, refname, db_start_idx)
    db_start_idx = db_start_idx + num_images
    
    del coords_table[:][:]

In [ ]:
"""
run2: random padding (use get_crop_image_by_coords_pad_random)
"""

# datadir = '/media/myungsungkwak/msdisk/docrv2_sroie/data/train_task1_crop/'
for folder_idx in range(4):
    
#     datadir = '/data/raw_data/train_task1_crop_split_sample_{}/'.format(folder_idx+1)
    datadir = '/data/raw_data/train_task1_crop_split_{}/'.format(folder_idx+1)
    print("[" + datadir + "]")
    
    img_files = glob(datadir+'*.jpg')
    txt_files = glob(datadir+'*.txt')

    crop_images = []
    coords_table = []

    datadb = create_database(folder_idx+1, "d2gan_train_task1_random_pad_sp")
    
    num_images = 1
    db_start_idx = 0
    for idx in range(len(img_files)):
        
        refname = os.path.basename(img_files[idx])[:-4]
        txt_filepath = os.path.join(datadir, refname+'.txt')
        
        coords_table.append(get_coords_from_image(txt_filepath))
        
        print("{}. {}".format(idx, img_files[idx]))
        print("{}. {}".format(idx, txt_filepath))
        
        num_images = len(coords_table[idx])
        crop_img, crop_coords, img_cnt = get_crop_image_by_coords_pad_random(img_files[idx],
                                                                             coords_table[idx],
                                                                             crop_size=(256,128),
                                                                             num_images=num_images
                                                                            )
        print("- orig imgs : {}, len(crimg) : {}, created imgs : {}, start_db_idx : {}, end_db_idx : {}".format(
            num_images,
            len(crop_img),
            img_cnt,
            db_start_idx,
            db_start_idx+img_cnt))

        update_database(datadb, crop_img, crop_coords, refname, db_start_idx)
        db_start_idx = db_start_idx + img_cnt

        del coords_table[:][:]
    print("********** complete {}".format(datadir))
print("...complete")

In [ ]:
"""
check heatmap
"""
"""
methods in D2GAN
"""
import cv2
def get_gaussian1d_heatmap(size=512, distanceRatio=1.5):
    # 1d gaussian heatmap along x axis
    v = np.abs(np.linspace(-size / 2, size / 2, num=size))
    x, y = np.meshgrid(v, v)
    g = np.sqrt(y ** 2)
    g *= distanceRatio / (size / 2)
    g = np.exp(-(1 / 2) * (g ** 2))
    g *= 255
    return g.clip(0, 255)

def compute_word_maps_no_space(heatmap, image_height, image_width, lines, descale=2, shiftx=0, shifty=0, paddx=0, paddy=0):

    textmap = np.zeros((image_height // descale, image_width // descale)).astype('float32')

    src = np.array([[0, 0], [heatmap.shape[1], 0], [heatmap.shape[1], heatmap.shape[0]], [0, heatmap.shape[0]]]).astype(
        'float32')

    for line in lines:
        for lind in range(len(line)):
            lvals = line[lind]
            x1 = lvals[0] - paddx - shiftx
            y1 = lvals[1] - paddy - shifty

            x2 = lvals[2] + paddx - shiftx
            y2 = lvals[3] - paddy - shifty

            x3 = lvals[4] + paddx - shiftx
            y3 = lvals[5] + paddy - shifty

            x4 = lvals[6] - paddx - shiftx
            y4 = lvals[7] + paddy - shifty
            c = lvals[8]

            character_points = np.array([[x1, y1], [x2, y2], [x3, y3], [x4, y4]]).astype(
                'float32') / descale

            MA = cv2.getPerspectiveTransform(
                src=src,
                dst=character_points,
            )

            textmap += cv2.warpPerspective(heatmap, MA, dsize=(textmap.shape[1], textmap.shape[0])).astype(
                'float32')

    emptymap = np.zeros(textmap.shape)
    return np.concatenate([textmap[..., np.newaxis], emptymap[..., np.newaxis], emptymap[..., np.newaxis]],
                            axis=2)

In [ ]:
def show_images_in_db(db_name,
                      db_base=common_variable['database']['DB_BASE'],
                      prefix='img',
                      show_imgs_count=10,
                      enable_db_info=False):
    target_db_path = os.path.join(db_base, db_name)
    target_db = open_env(target_db_path)

    if enable_db_info:
        print_env(target_db)
        print_db(target_db)

    imgs = read_bulk_data_from_db(target_db, prefix='img')
    imgs_len = len(imgs)
    # or
    # imgs_length = show_imgs_count

    rows = 5
    cols = 5
    axes = []
    fig = plt.figure(figsize=(30,30))
    
    idx = 0
    for idx in range(imgs_len):
        axes.append(fig.add_subplot(rows, cols, idx+1))
        plt.axis('off')
        plt.imshow(imgs[idx])
    
    fig.tight_layout()
    plt.show()

In [ ]:
"""
check map images
"""
datadb = open_env(path=common_variable['database']['DB_BASE'] + 'd2gan_train_task1_random_pad_sample_0001')
imgs = read_bulk_data_from_db(datadb, prefix='img')
char_c = read_bulk_data_from_db(datadb, prefix='char_c')
# print_db(datadb)

"""
show 10 images
"""
vimgs = []
for index in range(len(imgs)):
    img_size = imgs[index].size
    heatmap = get_gaussian1d_heatmap(distanceRatio=1.5)
    lines = char_c[index]
    for idx in range(len(lines)):
        lines[idx][0:8] = list(map(int, lines[idx][0:8]))
    mapimg = compute_word_maps_no_space(heatmap, img_size[1], img_size[0], [lines], descale=2)
    
    vimgs.append(imgs[index])
    vimgs.append(mapimg)


fig = plt.figure(figsize=(15, 15))
columns = 4
rows = 5
ax = []
for index in range(columns*rows):
    ax.append(fig.add_subplot(rows, columns, index+1) )
    ax[-1].set_title("index : "+str(index))
    plt.imshow(vimgs[index])
plt.axis('off')
plt.show()